In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F

In [2]:
device = 'cuda' if torch.cuda.is_available() else cpu
print(device)
block_size = 8
batch_size = 4
max_iter = 1000
learning_rate = 3e-4
eval_iters = 250
dropout = 0.2  #avoids overfitting by removing certain number of neurons

cuda


In [3]:
with open('wizard_of_oz.txt', 'r', encoding='utf-8')as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [4]:
string_to_int = { ch:i for i, ch in enumerate(chars) }
int_to_string = { i:ch for i, ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

encoded_hello = encode('hello')  
print(encoded_hello)

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

[61, 58, 65, 65, 68]
tensor([ 1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26, 49,
         0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,  0,
         0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1, 47,
        33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1, 36,
        25, 38, 28,  1, 39, 30,  1, 39, 50,  9])


In [5]:
print(decode(encoded_hello))

hello


In [6]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size,(batch_size,) )
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y
x,y = get_batch('train')
print('inputs:')
print(x.shape)
print(x)
print('targets:')
print(y)

inputs:
torch.Size([4, 8])
tensor([[69, 62, 60, 65, 58, 73,  9,  1],
        [57,  1, 54, 73,  1, 73, 61, 58],
        [ 1, 62, 73, 23,  1, 59, 68, 71],
        [ 9,  3,  1, 72, 54, 62, 57,  1]], device='cuda:0')
targets:
tensor([[62, 60, 65, 58, 73,  9,  1,  3],
        [ 1, 54, 73,  1, 73, 61, 58,  1],
        [62, 73, 23,  1, 59, 68, 71,  1],
        [ 3,  1, 72, 54, 62, 57,  1, 73]], device='cuda:0')


In [7]:
block_size =  8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):  #based on the current char predict the next char
    context = x[:t+1]
    target = y[t]
    print(f" when input is {context}, target is {target}")
    

 when input is tensor([1]), target is 1
 when input is tensor([1, 1]), target is 28
 when input is tensor([ 1,  1, 28]), target is 39
 when input is tensor([ 1,  1, 28, 39]), target is 42
 when input is tensor([ 1,  1, 28, 39, 42]), target is 39
 when input is tensor([ 1,  1, 28, 39, 42, 39]), target is 44
 when input is tensor([ 1,  1, 28, 39, 42, 39, 44]), target is 32
 when input is tensor([ 1,  1, 28, 39, 42, 39, 44, 32]), target is 49


In [8]:
import numpy as np 
import time 
print(np.__version__)

1.26.4


In [9]:
@torch.no_grad()   # makes sure torch doesnt use any gradient here
def estimate_loss():
    out={}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X,Y = get_batch(split)
            logits, loss = model(X,Y)
            losses[k]= loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [10]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index) #logits behaves like probability distribution of what we wanna predict
        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape # batch time channel
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)
        return logits, loss
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            #focus only on the last time step
            logits = logits[:, -1, :] # becomes B, C
            #apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1)# B, C
            #sample from the distribution
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next),dim=1) #(B, T+1)
            
        return index
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype= torch.long, device= device) #torch.long equivalent to int64
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)
    


WPph!(]BPiil
*fM;QIku tdF'UN5dLvGjTWM6[ET7pgs!?5&;'SfMMo8H8ujK&N5EYaunxF3lB !UPp2rq*92UN57kVd;V(7
?d8tR.1ti[em6d']'?ElVOy]IMwIrRV.ztwuMEUP ]g!2hQ-'U:)l14aRdtAN.dTLU;zdSdBw-7:y4]s'w-8seESd,dJxbKw91di1&dFC(B:1&C9'JRsg0Tcy9GY.WG;'IJJbuONAgT:Fp6LtII:sFp;CgnUp -8Nm;xxI64_2jrNfSWMoQX7]qMnEKR a1I"
fkHi7KXmsNB;qa'ZS(K5YR Pb)EIhlqZGjtH-IkRxWUppuvIJG
B5
*])KrNF3Y33.BRd&o(y)-6l"
v!4bu.,7g InuehjHz9pjg CcgdFN)lUR_50HvtXQg!4oRlmIZ.A
'
d'w-3G.bkHUjHN."M bo?mhmse0mk
YA2!4.:-W(w8KxoRpUH-Nhzrr0i6FIkcmaQuTFnVe'AR


In [14]:
#create a pytorch optimiser
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iter):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train-loss: {losses['train']:.3f}, val-loss: {losses['val']:.3f}")
    #sample a batch of data
    xb, yb = get_batch('train')
    #evaluate loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(f"Loss shape: {loss.shape}, Loss type: {type(loss)}")
print(f"Loss value: {loss.item()}")

step: 0, train-loss: 4.410, val-loss: 4.430
step: 250, train-loss: 4.376, val-loss: 4.367
step: 500, train-loss: 4.300, val-loss: 4.305
step: 750, train-loss: 4.251, val-loss: 4.276
Loss shape: torch.Size([]), Loss type: <class 'torch.Tensor'>
Loss value: 4.107603073120117


In [15]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


7]5Zg?_w-_SdxA&(RRRl[D."8(5f;[U*'
7.)dUBB'*R6]XZbB!ldsP? 69u?v2!jIEpX63i6C2yK5m,LxOy4mi2k,Zb)6Z-NERdqab"YU* ]q_FiF0Cs_9yN41nxdv9GHqoRdizUBMLG
aI5BKw:y)sRMYw _D2!-N4oXo26l&&QjExOp
YA(hADUCs&Q?TdI_P6j1OuWY8yEUez[3!x.,3Ydjq2'gIf5y[6
Z&C2J(oQIULLLa-14RzCB4VPp_5-*9V3He
TEco2C6ZbysymhEm]2hEdI:oX&Qh2rwCKppw5h7i0LvS2JdF5-HXSr0_k7cUBe0RW4omV!X:3.:
aV[noG
t_eefaYq;0C9y6]e1Q[Q.X9.H4ETLa0_hbWtli rHZima8
Y J_nh1nruls;qop*hIXHT!1bf[1I]HGDOiZZVi6SdYZEPLx'
q7 
TUHUBEwapFyvwaF5lq)Eccrf1nl"qp3GD.bE*HRk(yelm4NPig 
